In [1]:
#輸出圖檔名、圖檔大小############
%load_ext autotime
figure_name = "n_neighbors_10_min_dist_0.99_result"
dot_size = 3
length, width = 8, 12
n_neighbors = 10
min_dist = 0.99
n_components = 2
metric = 'euclidean'

time: 428 µs (started: 2021-06-07 17:23:13 +08:00)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

time: 15 s (started: 2021-06-07 17:23:13 +08:00)


In [3]:
data = pd.read_excel("./GTEx_NPC_data.xlsx", engine='openpyxl', header = None, index_col=0)
data = data.to_numpy()
label_data = pd.read_excel("./GTEx_NPC_data.xlsx", engine='openpyxl', header = None, index_col=None, usecols = "A")
label_data = label_data.to_numpy().flatten()

time: 16.3 s (started: 2021-06-07 17:23:28 +08:00)


In [4]:
row_sums = data.sum(axis=1)
normalize_data = data / row_sums[:, np.newaxis]

time: 2.72 ms (started: 2021-06-07 17:23:44 +08:00)


#### UMAP參數修改

In [ ]:
umap_x = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2, metric=metric).fit_transform(normalize_data)

In [ ]:
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

number = np.unique(label_data).shape[0]
cmap = plt.get_cmap('rainbow')
colors = [cmap(i) for i in np.linspace(0, 1, number)]
color_dic = {}
cancers = np.unique(label_data).tolist()
for i in range(number):
    color_dic[cancers[i]] = colors[i]
x_dot = umap_x[:,0]
y_dot = umap_x[:,1]

In [ ]:
dx = pd.DataFrame(x_dot, columns=['x'])
dy = pd.DataFrame(y_dot, columns=['y'])
df = pd.concat((dx, dy), axis=1)

In [ ]:
silhouette_avg = []
# 總共有30種tissues
for k in range(20,31):
    kmeans = KMeans(n_clusters=k).fit(df)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    # 輪廓係數法
    # S = (b-a)/max(a,b) 找出同群資料點內最近/不同群越分散的值
    # b為不同群之間的點平均距離、a為同群之間的距離
    silhouette_avg.append(silhouette_score(df, labels))
print(silhouette_avg)


In [ ]:
n_clusters = 20 + silhouette_avg.index(max(silhouette_avg))
kmeans = KMeans(n_clusters=n_clusters).fit(df)
labels = kmeans.labels_
centers = kmeans.cluster_centers_

In [ ]:
fig = plt.figure(figsize=(width,3*length), dpi=200)
ax = plt.subplot(311)
for g in np.unique(label_data):
	ix = np.where(label_data == g)
	ax.scatter(x_dot[ix], y_dot[ix], c = color_dic[g], label = g, s = dot_size)
ax.set_title('n_neighbors=' + str(n_neighbors) + ', min_dist=' + str(min_dist) + ', n_components=' + str(n_components) + ', metric=' + metric)
ax.legend(bbox_to_anchor=(1.05, 0), loc=3, borderaxespad=0)

plt.subplot(312)
plt.title("Silhouette score")
plt.plot(range(20,31), silhouette_avg)
plt.xticks(range(20,31))

ax = plt.subplot(313)
for i,g in zip(range(n_clusters),np.unique(label_data)):
	ax.scatter(df[labels==i].x, df[labels==i].y, c = color_dic[g], label = i, s = dot_size)
plt.scatter(centers.T[0], centers.T[1], c='black', marker='^', s=dot_size)
for j in range(centers.shape[0]):
    plt.text(centers.T[0][j], centers.T[1][j], str(j), fontdict={'color': 'black', 'weight': 'bold', 'size': 12})
ax.set_title("Kmeans = " + str(n_clusters) + " groups")
ax.legend(bbox_to_anchor=(1.05, 0), loc=3, borderaxespad=0)
plt.savefig("./result/%s.jpg" % figure_name, dpi=300, bbox_inches = 'tight')